In [1]:
import kfp
import kfp.components as comp
from kfp.v2 import dsl
from kfp.v2.dsl import component, Input, Output, OutputPath, Dataset, Model, ClassificationMetrics

In [2]:
kfp.__version__

'1.8.21'

In [3]:
# component 1
@component(packages_to_install=['loguru'])
def logg_some_stuff(input_text: str,
                    output_text: OutputPath(str)):
    from loguru import logger
    
    logger.info(f'The input text is {input_text}')

    out_text =  f'The awesome input text was {input_text}'

    with open(output_text, 'w') as output_file:
        output_file.write(out_text)

In [4]:
@dsl.pipeline(name='test-pipeline')
def kfp_hands_on_pipeline(initial_text: str,
                         gamma: float,
                         C: float):
    logger_task1 = logg_some_stuff(input_text=initial_text)